## Importing Libraries

In [3]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import torch.optim as optim

## Importing Data

In [5]:
def get_list(file_path):
    data = []
    with open(file_path, 'r') as file:
        for line in file:
            word, label = line.strip().split()
            data.append((word, int(label)))
    return data

In [6]:
def string_to_tensor(s):
    tensor  = torch.zeros(15, 26, dtype = torch.float32)
    for i, c in enumerate(s):
        tensor[i, ord(c) - ord('a')] = 1
    return tensor

In [7]:
def process_data(file_path):
    data = get_list(file_path)
    processed_data = [(string_to_tensor(word), float(class_id)) for word, class_id in data]
    return processed_data

In [8]:
train_data = process_data('training_set.txt')
TrainDataLoader = DataLoader(train_data, batch_size=256, shuffle=True)
test_data = process_data('test_set.txt')
TestDataLoader = DataLoader(test_data, batch_size=256, shuffle=False)

## Model Class

In [10]:
class MyRNN(nn.Module):
    
    def __init__(self, input_size, hidden_size, output_size, device):
        super(MyRNN, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        
        self.rnn_1 = nn.RNN(input_size, hidden_size, batch_first = True)
        self.rnn_2 = nn.RNN(hidden_size, hidden_size, batch_first = True)
        self.fcl = nn.Linear(hidden_size, output_size)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x):
        h0 = torch.zeros(1, x.size(0), self.hidden_size).to(device)
        out, _ = self.rnn_1(x, h0)
        out, _ = self.rnn_2(out, h0)
        out = self.fcl(out)
        out = self.sigmoid(out)
        return out

## Model Declaration

In [12]:
input_size = 26
hidden_size = 60
output_size = 1
device = ('cuda' if torch.cuda.is_available() else 'cpu')

In [13]:
model = MyRNN(input_size, hidden_size, output_size, device)
model.to(device)

MyRNN(
  (rnn_1): RNN(26, 60, batch_first=True)
  (rnn_2): RNN(60, 60, batch_first=True)
  (fcl): Linear(in_features=60, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)

## Eval function

In [15]:
def calculate_accuracy(model, dataloader):
    model.eval()
    correct = 0
    total = 0
    
    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            outputs = outputs[:, -1, :]
            predicted = (outputs > 0.5).float()
            labels = labels.view(-1, 1)
            
            correct += (predicted == labels).sum().item()
            total += labels.size(0)
    
        accuracy = (correct / total) * 100
        print(f'   Accuracy: {accuracy:.2f}%')

    return accuracy

In [16]:
start_accuracy = calculate_accuracy(model, TestDataLoader)

   Accuracy: 50.19%


## Training Loop

In [18]:
num_epochs = 500
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

In [19]:
for epoch in range(num_epochs):
    model.train()
    for inputs, labels in TrainDataLoader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        
        outputs = model(inputs)
        outputs = outputs[:, -1, :]
        outputs = outputs.view(-1)

        outputs = outputs.float()
        labels = labels.float()
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

    if ((epoch+1)%50 == 0):
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")
        calculate_accuracy(model, TestDataLoader)

Epoch [50/500], Loss: 0.1364
   Accuracy: 81.59%
Epoch [100/500], Loss: 0.1252
   Accuracy: 85.04%
Epoch [150/500], Loss: 0.1236
   Accuracy: 87.07%
Epoch [200/500], Loss: 0.0845
   Accuracy: 88.44%
Epoch [250/500], Loss: 0.0621
   Accuracy: 88.85%
Epoch [300/500], Loss: 0.0588
   Accuracy: 89.11%
Epoch [350/500], Loss: 0.0748
   Accuracy: 89.30%
Epoch [400/500], Loss: 0.0851
   Accuracy: 89.37%
Epoch [450/500], Loss: 0.0667
   Accuracy: 89.37%
Epoch [500/500], Loss: 0.0630
   Accuracy: 89.52%
